In [1]:
from wfOpto import *

## AL_0023

In [37]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-30\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.]
[-0.]
[0.1  0.25 0.5  0.75 1.   1.5  2.   2.5  3.  ]
[0.1]


In [38]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-30\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.]
[-0.]
[0.1  0.25 0.5  0.75 1.   1.5  2.   2.5  3.  ]
[0.1]


In [6]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-09\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[1. 4.]
[-0.]
[0.1 0.5 0.6 0.7]
[0.1]


In [8]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-11\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[1. 4.]
[-0.]
[0.1 0.5 1.  1.5 2.  2.5 3. ]
[0.1]


In [13]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-14\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[1. 4.]
[-0.]
[0.1 0.5 1.  1.5 2.  2.5 3. ]
[0.1]


In [14]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-15\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[1. 4.]
[-0.]
[1. 2. 3. 4. 5.]
[0.1]


In [15]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-23\3')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.]
[-0.   1.5  2.5]
[0.05 0.1  0.15 0.2  0.3  0.5  0.75 1.   1.5 ]
[0.1]


In [17]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-24\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.]
[-0.   1.5  2.5]
[0.1  0.25 0.5  0.75 1.   1.5  2.   2.5  3.  ]
[0.1]


In [20]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-25\2')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.]
[-0.   1.5  2.5]
[0.1  0.25 0.5  0.75 1.   1.5  2.   2.5  3.  ]
[0.1]


In [21]:
serverPath = Path(r'Z:\Subjects\AL_0023\2023-08-30\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.]
[-0.]
[0.1  0.25 0.5  0.75 1.   1.5  2.   2.5  3.  ]
[0.1]


## ZYE_0069

In [22]:
serverPath = Path(r'Z:\Subjects\ZYE_0069\2023-07-14\4')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[0.]
[0.]
[3.2]
[0.5]


In [40]:
serverPath = Path(r'Z:\Subjects\ZYE_0069\2023-07-14\6')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

FileNotFoundError: [Errno 2] No such file or directory: 'Z:\\Subjects\\ZYE_0069\\2023-07-14\\6\\galvoXPositions.npy'

In [29]:
# this one is messy since it switches halfway through
serverPath = Path(r'Z:\Subjects\ZYE_0069\2023-07-17\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.   3.3]
[0.]
[0.1 0.4 0.5 0.7 1.1 2.2 3.2]
[0.01 0.05 0.1  0.2  0.5 ]


In [31]:
serverPath = Path(r'Z:\Subjects\ZYE_0069\2023-07-19\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[0.  3.3]
[0.  3.3]
[0.5 1.  1.5 2.  3.2]
[0.1]


In [32]:
serverPath = Path(r'Z:\Subjects\ZYE_0069\2023-08-16\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.01 -0.  ]
[-0.01 -0.  ]
[0.1 0.2 0.5 0.7 1.  1.2 1.5 1.7 2. ]
[0.1]


In [35]:
serverPath = Path(r'Z:\Subjects\ZYE_0069\2023-08-22\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.]
[-0.]
[0.05 0.1  0.15 0.2  0.3  0.5  0.75 1.   1.5 ]
[0.1]


## AL_0027

In [36]:
serverPath = Path(r'Z:\Subjects\AL_0027\2023-10-24\1')
galvoX = np.squeeze(np.load(serverPath/'galvoXPositions.npy'))
galvoY = np.squeeze(np.load(serverPath/'galvoYPositions.npy'))
laserPowers = np.squeeze(np.load(serverPath/'laserPowers.npy'))
laserOn = np.squeeze(np.load(serverPath / 'laserOnTimes.npy'))
laserOff = np.squeeze(np.load(serverPath / 'laserOffTimes.npy'))
pulseLengths = []
for count,time in enumerate(laserOff):
        length = laserOff[count]-laserOn[count]
        length = round(length,2)
        pulseLengths.append(length)
pulseLengths = np.array(pulseLengths)
print(np.unique(galvoX))
print(np.unique(galvoY))
print(np.unique(laserPowers))
print(np.unique(pulseLengths))

[-0.]
[-0.]
[0.1  0.25 0.5  0.75 1.   1.5  2.   2.5  3.  ]
[0.1]
